#  Lab: Training on multiple GPUs with `gluon`

This lab demonstrates the concepts of how to split up the training of a model across multiple GPUs using Gluon. Dat a parallelism will be used where each batch is split into equal portions, a forward and backward pass is performed, and the gradients are summed and the parameters are updated. A complete copy of all the parameters is present on each GPU.

This lab has been adapted from https://gluon.mxnet.io/chapter07_distributed-learning/multiple-gpus-gluon.html.

This lab has been tested using a ml.p3.8xlarge SageMaker notebook instance. It requires an instance with multiple GPUs.

The key steps are:

* Choose “local“ or “device“ kvstore

* Initialise the parameters and copy all of them to each GPU
* Split up the batch into portions and copy each portion onto a GPU
* Run forward and backward

The following steps are run automatically by Gluon when parameters on multiple devices are detected:
* Sum the gradients across all GPUs and broadcast to all GPUs
* Update the weights

Start by defining a simple convolutional neural network for image classification:

In [ ]:
import mxnet as mx
from mxnet import nd, gluon, autograd
from time import time
net = gluon.nn.Sequential(prefix='cnn_')
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20, kernel_size=3, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=(2,2), strides=(2,2)))
    net.add(gluon.nn.Conv2D(channels=50, kernel_size=5, activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=(2,2), strides=(2,2)))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(128, activation="relu"))
    net.add(gluon.nn.Dense(10))
    
loss = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
class Profile:
    
    checkpoints = {}
    
    def __init__(self,silent=False):
        self.silent = silent
    
    def start(self,event_name='default'):
        self.checkpoints[event_name] = time()
        return self
        
    def stop(self,event_name='default',print_out=True):
        current_time = time()
        checkpoint = self.checkpoints[event_name]
        period_seconds = current_time - checkpoint
        
        if print_out and not self.silent:
            print('{} {:.4f} sec'.format(event_name,period_seconds))
        
        return period_seconds

## Initialize on multiple devices

Gluon supports initialization of network parameters over multiple devices. This is done by passing in an array of device contexts, instead of a single context.
When we pass in an array of contexts, the parameters are initialized 
to be identical across all of our devices.

In [ ]:
GPU_COUNT = 4
ctx = [mx.cpu(i) for i in range(GPU_COUNT)]
net.initialize(ctx=ctx)

Each batch of input data is split into parts (one for each GPU) 
by calling `gluon.utils.split_and_load(batch, ctx)`.
The `split_and_load` function also loads each part onto the appropriate device context. 

When the forward and backwards passes are computed later, this is executed on the device with the version of the parameters which have been stored there.

In [ ]:
from mxnet.test_utils import get_mnist

mnist = get_mnist()
batch = mnist['train_data'][0:GPU_COUNT*2, :]
data = gluon.utils.split_and_load(batch, ctx)
print(net(data[0]))
print(net(data[1]))

At any time, we can access the version of the parameters stored on each device. 
Our weights may not actually be initialized when we call `initialize` because the parameter shapes may not yet be known. In these cases, initialization is deferred pending shape inference. 

In [ ]:
weight = net.collect_params()['cnn_conv0_weight']

for c in ctx:
    print('=== channel 0 of the first conv on {} ==={}'.format(
        c, weight.data(ctx=c)[0]))
    

Similarly, we can access the gradients on each of the GPUs. Because each GPU gets a different part of the batch (a different subset of examples), the gradients on each GPU vary. 

In [ ]:
def forward_backward(net, data, label, p):
    with autograd.record():
        p.start('forward')
        losses = [loss(net(X), Y) for X, Y in zip(data, label)]
        p.stop('forward')
        
    p.start('backward')
    
    for l in losses:
        l.backward()
    
    p.stop('backward')
        
label = gluon.utils.split_and_load(mnist['train_label'][0:4], ctx)
forward_backward(net, data, label, Profile())

for c in ctx:
    print('=== grad of channel 0 of the first conv2d on {} ==={}'.format(
        c, weight.grad(ctx=c)[0]))

## Put all things together

Now we can implement the remaining functions. Most of them are the same as [when we did everything by hand](./chapter07_distributed-learning/multiple-gpus-scratch.ipynb); one notable difference is that if a `gluon` trainer recognizes multi-devices, it will automatically aggregate the gradients and synchronize the parameters. 

In [ ]:
from mxnet.io import NDArrayIter

def train_batch(batch, ctx, net, trainer, p):
    # split the data batch and load them on GPUs
    p.start('split_and_load')
    data = gluon.utils.split_and_load(batch.data[0], ctx)
    label = gluon.utils.split_and_load(batch.label[0], ctx)
    p.stop('split_and_load')
    # compute gradient
    p.start('forward_backward')
    forward_backward(net, data, label, p)
    p.stop('forward_backward')
    # update parameters
    p.start('update')
    trainer.step(batch.data[0].shape[0])
    p.stop('update')
    
def valid_batch(batch, ctx, net):
    data = batch.data[0].as_in_context(ctx[0])
    pred = nd.argmax(net(data), axis=1)
    return nd.sum(pred == batch.label[0].as_in_context(ctx[0])).asscalar()    

def run(num_gpus, batch_size, lr):    
    
    p = Profile(silent=True)
    
    # the list of GPUs will be used
    ctx = [mx.gpu(i) for i in range(num_gpus)]
    print('Running on {}'.format(ctx))
    
    # data iterator
    mnist = get_mnist()
    train_data = NDArrayIter(mnist["train_data"], mnist["train_label"], batch_size)
    valid_data = NDArrayIter(mnist["test_data"], mnist["test_label"], batch_size)
    print('Batch size is {}'.format(batch_size))
    
    net.initialize(force_reinit=True, ctx=ctx)
    # Here the kvstore can be set to 'local' where the gradients are summed and synced on the cpu
    # or 'device' on the GPUs. If 'device' is selected mxnet uses GPU to GPU comms where possible.
    trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': lr}, kvstore='local')
    
    for epoch in range(5):
        # train
        p.start('epoch')
        train_data.reset()
        for batch in train_data:
            train_batch(batch, ctx, net, trainer, p)
        nd.waitall()  # wait until all computations are finished to benchmark the time
        print('Epoch %d, training time = %.1f sec'%(epoch, p.stop('epoch',False)))
        
        # validating
        valid_data.reset()
        correct, num = 0.0, 0.0
        for batch in valid_data:
            correct += valid_batch(batch, ctx, net)
            num += batch.data[0].shape[0]                
        print('         validation accuracy = %.4f'%(correct/num))
                
run(GPU_COUNT, 512*GPU_COUNT, .3) # a larger batch size is used so each GPU has enough data 
run(1, 512, .3)

## Conclusion

We have successfully run multi-gpu training using gluon. Experiment with the GPU_COUNT and batch_size. Set silent = False when instantiating the Profile class to see times of each training step.